# Reappraisal Training For Linguistic Distancing and Emotion Regulation


## Setup
```bash
> pipenv shell  #Generates a new virtual environment based on Pipfile
> pipenv install # Installs the packages in Pipfile.lock (Use --dev) to also install dev packages
```
## Included Datasets
- LDHII 
- 
- Emobank
**Sources**
-  [Sentiment Analysis Text Classification Tutorial](https://www.youtube.com/watch?v=8N-nM3QW7O0)
- [Using Catalyst for Training Organization](https://github.com/catalyst-team/catalyst)



In [3]:
# pip install transformers datasets nltk


In [1]:
import os
import numpy as np
import pandas as pd
import torch
from datasets import ReadInstruction

# Enable GPU usage, if we can.
if torch.cuda.is_available():
    print("Enabling GPU usage")
    device = torch.device("cuda:0")
    print(device)
    IS_GPU = True
else:
    print("No GPU available, running on CPU")
    device = torch.device("cpu") # Note: macOS incompatible with NVIDIA GPUs
    IS_GPU = False
    

PRETRAINED_MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'

No GPU available, running on CPU


### LDH Dataset Imports

In [2]:
from src.LDHData import LDHData

data = LDHData()
data.load_training_data()
data.load_eval_data()

Training data loaded from disk.
Evaluation data loaded from disk.


In [18]:
data.train_dataset

DatasetDict({
    far: Dataset({
        features: ['response', 'score'],
        num_rows: 13470
    })
    obj: Dataset({
        features: ['response', 'score'],
        num_rows: 13470
    })
})

In [4]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained(PRETRAINED_MODEL_NAME)  
# Wrap tokenizer.
def tokenize(x):
    tokenized = tokenizer(x, add_special_tokens=True, padding="max_length", max_length=150)
    return tokenized
    
encoded_train = data.train_dataset['far'].map(
    lambda ds: tokenize(ds['response']), batched=True, batch_size=16
)
encoded_train.set_format(type='torch', output_all_columns=True)

Loading cached processed dataset at /home/danielpham/reapp/reappraisal-model/src/training/far/cache-6c98bb4bcbc39ba4.arrow


1347

In [23]:
from transformers import DistilBertModel, TrainingArguments, Trainer
from src.ReappModel import ReappModel
# Define the parameters under which the model will be trained.
# By default, uses an AdamW optimizer w/ linear warmup.
model = ReappModel(DistilBertModel.from_pretrained(PRETRAINED_MODEL_NAME))

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,
    args=training_args,                  
    train_dataset=encoded_train,
    #eval_dataset=encoded_eval         
)

In [6]:
trainer.train()

Step,Training Loss


KeyboardInterrupt: 